In [90]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Dropout, Activation
from keras.models import Sequential
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [121]:
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [13]:
train_data = pd.read_csv('../data/clean/emotion_anger_regression.csv')
train_data.head()

,content,sentiment_value
0,it makes me so fucking irate jesus. nobody is ...,0.750
1,Lol Adam the Bull with his fake outrage...,0.417
2,@THATSSHAWTYLO passed away early this morning ...,0.354
3,@Kristiann1125 lol wow i was gonna say really?...,0.438
4,I need a \xf0\x9f\x8d\xb1sushi date\xf0\x9f\x8...,0.271


In [14]:
train_data['target'] = train_data.sentiment_value.astype('category').cat.codes

In [15]:
MAX_LENGTH = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data.content.values)
post_seq = tokenizer.texts_to_sequences(train_data.content.values)
post_seq_padded = pad_sequences(post_seq, maxlen=MAX_LENGTH)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(post_seq, train_data.target, test_size=0.05)

In [17]:
num_classes = max(y_train) + 1
tokenizer = Tokenizer(num_words=MAX_LENGTH)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [18]:
model = Sequential()
model.add(Dense(512, input_shape=(MAX_LENGTH,)))
model.add(Dense(256, input_shape=(MAX_LENGTH/2,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.metrics_names)

['loss', 'acc']


In [19]:
batch_size = 64
epochs = 5

In [20]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.1)
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 1453 samples, validate on 162 samples
Epoch 1/5
1453/1453 [==============================] - 0s 261us/step - loss: 5.0726 - acc: 0.0262 - val_loss: 4.5106 - val_acc: 0.0247
Epoch 2/5
1453/1453 [==============================] - 0s 76us/step - loss: 4.2546 - acc: 0.0537 - val_loss: 4.3726 - val_acc: 0.0370
Epoch 3/5
1453/1453 [==============================] - 0s 72us/step - loss: 3.9695 - acc: 0.0881 - val_loss: 4.3472 - val_acc: 0.0432
Epoch 4/5
1453/1453 [==============================] - 0s 79us/step - loss: 3.8154 - acc: 0.0929 - val_loss: 4.3246 - val_acc: 0.0494
Epoch 5/5
85/85 [==============================] - 0s 35us/step
Test loss: 4.310807631997501
Test accuracy: 0.0705882357323871
